<a href="https://colab.research.google.com/github/jowu-brunonian/Brown-University-DSIO-2000/blob/main/Assessment_3_Optimizing_a_linear_model_using_gradient_descent_due_end_of_Module_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source of data: Electric Vehicle Population Data https://catalog.data.gov/dataset/electric-vehicle-population-data

Based on the available information in the notebook, the data is sourced from data.gov and contains registration data from based on registered VIN numbers of electric vehicles within the state of Washington

Downloaded and imported CSV into Google Drive for Public Viewing: https://docs.google.com/spreadsheets/d/1yh6j5HBTlhZzFQy-7JixcnW0CoILKn4x3ioi_LbqmGc/edit?gid=443715183#gid=443715183

In [ ]:
#importing the gdown library to import the excel file
import gdown

#importing pandas to create a dataframe, statsmodel to perform linear algebra, and numpy for numpy packages
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [ ]:
#Calling gdown and the file id
!gdown 1yh6j5HBTlhZzFQy-7JixcnW0CoILKn4x3ioi_LbqmGc

Downloading...
From (original): https://drive.google.com/uc?id=1yh6j5HBTlhZzFQy-7JixcnW0CoILKn4x3ioi_LbqmGc
From (redirected): https://docs.google.com/spreadsheets/d/1yh6j5HBTlhZzFQy-7JixcnW0CoILKn4x3ioi_LbqmGc/export?format=xlsx
To: /content/Electric_Vehicle_Population_Data.xlsx
21.7MB [00:00, 119MB/s]


**Predictor feature (x):** Electric Range

**Response feature (y):** Base MSRP


To find the optimal values of B₀ and B₁, we set the partial derivatives of the cost function to zero:

Equation 1: ∂SSR/∂B₀ = −∑(yᵢ − (B₀ + B₁xᵢ)) = 0

Equation 2: ∂SSR/∂B₁ = −∑(yᵢ − (B₀ + B₁xᵢ)) · xᵢ = 0

In [ ]:
#This file contains the 'Base MSRP', and 'Electric Range' data.
#define a dataframe named 'EVehicles'
#using pd.read_excel to read in the file imported from previous step
#input parameter is the location of the file after calling gdown

Evehicles=pd.read_excel('/content/Electric_Vehicle_Population_Data.xlsx')

In [ ]:
import autograd.numpy as np
from autograd import grad
from tabulate import tabulate

# Step 1: Define 5 rows of data
# These are the observed values from the dataset
x_predictor = np.array([84, 114, 107, 140, 238])
y_observed = np.array([29995, 36570, 37995, 42250, 49995])

# Step 2: Define the SSR (Sum of Squared Residuals) cost function
def ssr_cost_function(b0, b1):
    y_predicted = b0 + b1 * x_predictor
    residuals = y_observed - y_predicted
    ssr = np.sum(residuals ** 2)
    return ssr

#Identifying the Cost Function
print("Initial Value of Sum of Squared Residuals (SSR) cost function:")
print(ssr_cost_function(current_b0, current_b1))


Initial Value of Sum of Squared Residuals (SSR) cost function:
7908219550.0


In [ ]:
# Step 3: Get partial derivatives
derivative_wrt_b0 = grad(ssr_cost_function, argnum=0)
derivative_wrt_b1 = grad(ssr_cost_function, argnum=1)

# Step 4: Choose initial values
#choose 0.0 (intercept) and 1.0 (shope), respectively, as arbitrary starting points for gradient descent
current_b0 = 0.0
current_b1 = 1.0

# Step 5: Calculate initial partial derivatives
partial_d_b0 = derivative_wrt_b0(current_b0, current_b1)
partial_d_b1 = derivative_wrt_b1(current_b0, current_b1)

print(f"Initial ∂SSR/∂B0: {partial_d_b0:.4f}")
print(f"Initial ∂SSR/∂B1: {partial_d_b1:.4f}")


Initial ∂SSR/∂B0: -392244.0000
Initial ∂SSR/∂B1: -56920180.0000


In [ ]:
from tabulate import tabulate

# Step 6: Perform 5 iterations of intercept updates

# Initialize lists to store values and define learning rate
# current_b0 and partial_d_b0 are available from the previous step
intercept_values = [current_b0]
partial_b0_values = [partial_d_b0]
learning_rate = 0.095  # An arbitrary learning rate chosen for starting point

for i in range(5):
    new_b0 = intercept_values[-1] - learning_rate * partial_b0_values[-1]
    new_partial = derivative_wrt_b0(new_b0, current_b1)
    intercept_values.append(new_b0)
    partial_b0_values.append(new_partial)

# Logic to create and print the table
if not intercept_values:
    generated_table = "No data provided to create a table."
else:
    # Prepare data for tabulate: list of lists, where each inner list is a row
    table_data = []
    for i in range(len(intercept_values)):
        table_data.append([intercept_values[i], partial_b0_values[i]])

    # Define table headers
    headers = ["Intercept Value", "Partial Derivative(∂SSR/∂B0)"]

    # Use tabulate to create a nicely formatted table
    table_string = tabulate(table_data, headers=headers, tablefmt="grid", floatfmt=".4f")
    generated_table = table_string

print("Intercept updates (5 iterations):")
print("---------------------------------")
print(generated_table)

Intercept updates (5 iterations):
---------------------------------
+-------------------+--------------------------------+
|   Intercept Value |   Partial Derivative(∂SSR/∂B0) |
+===================+================================+
|            0.0000 |                   -392244.0000 |
+-------------------+--------------------------------+
|        37263.1800 |                    -19612.2000 |
+-------------------+--------------------------------+
|        39126.3390 |                      -980.6100 |
+-------------------+--------------------------------+
|        39219.4970 |                       -49.0305 |
+-------------------+--------------------------------+
|        39224.1548 |                        -2.4515 |
+-------------------+--------------------------------+
|        39224.3877 |                        -0.1226 |
+-------------------+--------------------------------+


In [ ]:
#Step 7: scikit-learn Comparison
from sklearn.linear_model import LinearRegression
import numpy as np

# Define your data
x = np.array([84, 114, 107, 140, 238]).reshape(-1, 1)
y = np.array([29995, 36570, 37995, 42250, 49995])

# Fit the model
model = LinearRegression()
model.fit(x, y)

# Get coefficients
print("Intercept (B0):", model.intercept_)
print("Slope (B1):", model.coef_[0])


Intercept (B0): 23436.00725400078
Slope (B1): 116.58120604684643


In [ ]:
# Step 8: Perform 5 iterations of slope updates

# Use final intercept from previous loop
#Experimented with different learning_rate values
#reducing the learning rate to 0.000001, the slope updates stabilized and began converging toward a minimum.
final_b0 = intercept_values[-1]
initial_b1 = 0.0  # Start slope at zero
learning_rate = 0.000001

# Initialize lists
slope_values = [initial_b1]
partial_b1_values = [derivative_wrt_b1(final_b0, initial_b1)]

# Perform 5 iterations
for i in range(5):
    new_b1 = slope_values[-1] - learning_rate * partial_b1_values[-1]
    new_partial = derivative_wrt_b1(final_b0, new_b1)
    slope_values.append(new_b1)
    partial_b1_values.append(new_partial)

# Create and print slope update table
table_data_b1 = list(zip(slope_values, partial_b1_values))
headers_b1 = ["Slope Value (B1)", "Partial Derivative (∂SSR/∂B1)"]

print("\nSlope updates (5 iterations):")
print("------------------------------")
print(tabulate(table_data_b1, headers=headers_b1, tablefmt="grid", floatfmt=".4f"))



Slope updates (5 iterations):
------------------------------
+--------------------+---------------------------------+
|   Slope Value (B1) |   Partial Derivative (∂SSR/∂B1) |
+====================+=================================+
|             0.0000 |                   -3555156.3439 |
+--------------------+---------------------------------+
|             3.5552 |                   -2789055.7034 |
+--------------------+---------------------------------+
|             6.3442 |                   -2188042.0898 |
+--------------------+---------------------------------+
|             8.5323 |                   -1716540.8999 |
+--------------------+---------------------------------+
|            10.2488 |                   -1346643.5014 |
+--------------------+---------------------------------+
|            11.5954 |                   -1056455.2933 |
+--------------------+---------------------------------+


**Initial Cost Function Value:**

7,908,219,550.0 18

**Final Intercept ($\mathbf{B_0}$) after 5 Iterations:**

39,224.3877 19

**OLS Solution for Comparison:**

Intercept ($\mathbf{B_0}$): 23,436.007

Slope ($\mathbf{B_1}$): 116.581 22